In [ ]:
import os
import sys
import django
import logging
import itertools
import pandas as pd
import numpy as np
from tqdm import tqdm
from django.db.models import Count
from sklearn.metrics import mean_squared_error
sys.path.insert(0, "/Users/victornguyen/Sites/07.book_management/")
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "book_management.settings")
django.setup()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
import django
import time
from main_site.models import Rating
import rs_system.recommender as rs

logger = logging.getLogger('Evaluation')

In [ ]:
ratings_count = Rating.objects.all().count()
test_data_length = round(ratings_count * 10 /100)
all_ratings_df = pd.DataFrame(list(Rating.objects.all().values()))
logger.debug('{} ratings available'.format(ratings_count))
all_ratings_df.head()
all_ratings_df_columns = list(all_ratings_df.columns.values)
all_ratings_df_columns

In [ ]:
user_ids = Rating.objects.filter(type='calculate') \
    .values('user_id') \
    .annotate(title_count=Count('title_id')) \
    .order_by('-title_count')
print(user_ids.query)
title_ids = Rating.objects.filter(type='calculate') \
    .values('title_id') \
    .annotate(user_count=Count('user_id')) \
    .order_by('-user_count')
user_ids_df = pd.DataFrame.from_records(user_ids)
title_ids_df = pd.DataFrame.from_records(title_ids)
user_ids_df.head(5)

In [ ]:
from sklearn.model_selection import KFold 

In [ ]:
kf = KFold(n_splits=10)
for train, test in kf.split(all_ratings_df):
    train_data = np.array(all_ratings_df)[train]
    print("Train_data")
    print(train_data)
    test_data = np.array(all_ratings_df)[test]
    print("Test_data")
    print(test_data)
    print(len(train_data))
    print(len(test_data))
    test_pd = pd.DataFrame(test_data)
    test_pd.columns = all_ratings_df_columns
    print(test_pd)
    print("=====================")
    break
test_pd

In [ ]:
# all_ratings_df = pd.merge(all_ratings_df, user_ids_df, left_on='user_id', right_on='user_id')
# all_ratings_df = pd.merge(all_ratings_df, title_ids_df, left_on='title_id', right_on='title_id')
# all_ratings_df.head(5)

In [ ]:
count = 0
test_data = list()
for row in tqdm(user_ids_df.itertuples()):
    if row.title_count == 1:
        break
    if row.title_count > 10:
        limit_row = 3
    elif row.title_count > 5:
        limit_row = 2
    else:
        limit_row = 1
    test_data.extend(list(Rating.objects.filter(user_id=row.user_id).order_by('-rating').values()[:limit_row]))
    count +=limit_row
    if count == test_data_length:
        break
test_data_df =  pd.DataFrame(test_data)
test_data_id = list(test_data_df.id)
test_data_id
# [41, 58, 23, 54, 18, 1, 130, 83, 139, 105, 21, 48, 51, 28

In [ ]:
train_data_df = pd.DataFrame(list(Rating.objects.exclude(id__in=test_data_id).values()))
rating_df = pd.DataFrame(list(Rating.objects.all().values()))
train_data_df

In [ ]:
len(test_data)

In [ ]:
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mean_squared_error(y_true, y_pred)

In [ ]:
train_data_df

In [ ]:
test_data_df

In [ ]:
res_nb = nb_rs.RecommendationNB()
rating_matrix, item_rating_matrix = res_nb.get_rating_matrix()
mean_centered_rating_matrix = res_nb.get_mean_centered_ratings_matrix(rating_matrix)
item_mean_centered_rating_matrix = res_nb.get_mean_centered_ratings_matrix(item_rating_matrix)

user_similarity_matrix = res_nb.get_similarity_matrix(rating_matrix)
item_similarity_matrix = res_nb.get_similarity_matrix(item_rating_matrix)

In [ ]:
rating_matrix[8]

In [ ]:
y_true = []
y_pred = []
for row in tqdm(test_data_df.itertuples()):
    y_true.append(float(row.rating))
#     predict_result = res_nb.predict(row.user_id, row.title_id, rating_matrix, mean_centered_rating_matrix, user_similarity_matrix)
    y_pred.append(rating_matrix[row.user_id,row.title_id])
print(y_true)
print(y_pred)

In [ ]:
kf = KFold(n_splits=10)
for train, test in kf.split(all_ratings_df):
    # Train data
    train_data = np.array(all_ratings_df)[train]
    train_data_df = pd.DataFrame(train_data)
    train_data_df.columns = all_ratings_df_columns

    # Test data
    test_data = np.array(all_ratings_df)[test]
    test_data_df = pd.DataFrame(test_data)
    test_data_df.columns = all_ratings_df_columns

    # Initialize RS
    res_nb = nb_rs.RecommendationNB(train_data)
    y_true = []
    y_pred = []
    for row in tqdm(test_data_df.itertuples()):
        y_true.append(float(row.rating))
        predict_result = res_nb.predict(row.user_id - 1, row.title_id - 1, res_nb.rating_matrix,
                                            res_nb.mean_centered_ratings_matrix,
                                            res_nb.user_similarity_matrix)
        y_pred.append(predict_result)
    print(y_true)
    print(y_pred)
    print("=====================")
    break